##Retrieve data from MongoDB

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
db = client.RawDataDb
collection = db.AqiData
cursor = collection.find({})
document_list_aqi = []
for document in cursor:
    document_list_aqi.append(document)

In [4]:
collection = db.WeatherData
cursor = collection.find({})
document_list_weather = []
for document in cursor:
    document_list_weather.append(document)

In [5]:
collection = db.AqiAPIData
cursor = collection.find({})
document_list_aqiAPI = []
for document in cursor:
    document_list_aqiAPI.append(document)

In [6]:
collection = db.TrafficData
cursor = collection.find({})
document_list_traffic = []
for document in cursor:
    document_list_traffic.append(document)

In [7]:
collection = db.GeolocAqiStations
cursor = collection.find({})
document_list_geocoding = []
for document in cursor:
    document_list_geocoding.append(document)

##Data quality on the single collections of data

In [8]:
import pandas as pd
weather_df = pd.DataFrame.from_records(document_list_weather)
weather_df = weather_df.drop('_id', axis=1)

In [9]:
#completeness
m = len(weather_df)
n = len(list(weather_df.columns))
#table completeness
null_table = 0
for col in weather_df.columns:
    null_table += weather_df[col].isnull().sum()
table_completeness = 1-(null_table/(m*n))    
print('Table completeness: ', table_completeness, '\n\n')

#attribute completeness
null_attributes_completeness = []
for col in weather_df.columns:
    attr_compl = 1-(weather_df[col].isnull().sum()/m)
    null_attributes_completeness.append(attr_compl)

column_list = weather_df.columns
attribute_completeness_weather = {'attributes':list(column_list), 
                                  'attribute completeness':null_attributes_completeness}
attribute_completeness_weather_df = pd.DataFrame(attribute_completeness_weather, )
attribute_completeness_weather_df

Table completeness:  0.8418167139182516 


,attributes,attribute completeness
0,temp,1.000000
1,dwpt,1.000000
2,rhum,1.000000
3,prcp,1.000000
4,snow,0.002441
5,wdir,1.000000
6,wspd,1.000000
7,wpgt,0.941176
8,pres,1.000000
9,tsun,0.000000


In [10]:
aqi_scraping_df = pd.DataFrame.from_records(document_list_aqi)
aqi_scraping_df = aqi_scraping_df.drop('_id', axis=1)

In [11]:
#completeness
m = len(aqi_scraping_df)
n = len(list(aqi_scraping_df.columns))
#table completeness
null_table = 0
for col in aqi_scraping_df.columns:
    if col == 'aqi':
        null_table += (aqi_scraping_df[col] == 'no data').sum()
    if col == 'aqi_level':
        null_table += (aqi_scraping_df[col] == 'no data').sum()
    if col == 'pressure' or col=='time' or col=='station id' or col=='wind'or col=='pm 1':
        null_table += aqi_scraping_df[col].isnull().sum()
    else:
        null_table += (aqi_scraping_df[col] == '-').sum()
table_completeness = 1-(null_table/(m*n))    
print('Table completeness: ', table_completeness, '\n\n')

#attribute completeness
null_attributes_completeness = []
for col in aqi_scraping_df.columns:
    if col == 'aqi':
        attr_compl = 1-(((aqi_scraping_df[col] == 'no data').sum())/m)
    if col == 'aqi_level':
        attr_compl = 1-(((aqi_scraping_df[col] == 'no data').sum())/m)
    if col == 'pressure' or col=='time' or col=='station id' or col=='wind' or col=='pm 1':
        attr_compl = 1-(aqi_scraping_df[col].isnull().sum()/m)
    else:
        attr_compl = 1-(((aqi_scraping_df[col] == '-').sum())/m)
        
    null_attributes_completeness.append(attr_compl)

column_list = aqi_scraping_df.columns
attribute_completeness_aqi_sc = {'attributes':list(column_list), 
                                  'attribute completeness':null_attributes_completeness}
attribute_completeness_aqi_sc_df = pd.DataFrame(attribute_completeness_aqi_sc)
attribute_completeness_aqi_sc_df

Table completeness:  0.868521992328583 


,attributes,attribute completeness
0,station id,1.000000
1,time,1.000000
2,aqi,1.000000
3,aqi_level,1.000000
4,pm 2.5,0.941176
5,pm 10,0.941176
6,temp,0.941176
7,r.h,0.941176
8,pressure,0.764706
9,wind,1.000000


In [12]:
aqi_api_df = pd.DataFrame.from_records(document_list_aqiAPI)
aqi_api_df = aqi_api_df.drop('_id', axis=1)

In [13]:
#completeness
m = len(aqi_api_df)
n = len(list(aqi_api_df.columns))
#table completeness
null_table = 0
for col in aqi_api_df.columns:
    null_table += aqi_api_df[col].isnull().sum()
table_completeness = 1-(null_table/(m*n))    
print('Table completeness: ', table_completeness, '\n\n')

#attribute completeness
null_attributes_completeness = []
for col in aqi_api_df.columns:
    attr_compl = 1-(aqi_api_df[col].isnull().sum()/m)
    null_attributes_completeness.append(attr_compl)

column_list = aqi_api_df.columns
attribute_completeness_aqi_api = {'attributes':list(column_list), 
                                  'attribute completeness':null_attributes_completeness}
attribute_completeness_aqi_api_df = pd.DataFrame(attribute_completeness_aqi_api)
attribute_completeness_aqi_api_df

Table completeness:  1.0 


,attributes,attribute completeness
0,us_aqi,1.0
1,us_aqi_pm2_5,1.0
2,us_aqi_pm10,1.0
3,us_aqi_nitrogen_dioxide,1.0
4,us_aqi_carbon_monoxide,1.0
5,us_aqi_ozone,1.0
6,us_aqi_sulphur_dioxide,1.0
7,station,1.0
8,datetime,1.0


In [14]:
location_list = []
for d in document_list_traffic:
    location_list.append(d['location'])
flow_list = []
for d in document_list_traffic:
    flow_list.append(d['currentFlow'])
datetime_list = []
for d in document_list_traffic:
    datetime_list.append(d['datetime'])

location_df = pd.DataFrame.from_records(location_list)
flow_df = pd.DataFrame.from_records(flow_list)
traffic_df = pd.merge(location_df, flow_df, right_index=True, left_index=True)
traffic_df['datetime'] = datetime_list

In [15]:
#completeness
m = len(traffic_df)
n = len(list(traffic_df.columns))
#table completeness
null_table = 0
for col in traffic_df.columns:
    null_table += traffic_df[col].isnull().sum()
table_completeness = 1-(null_table/(m*n))    
print('Table completeness: ', table_completeness, '\n\n')
#attribute completeness
null_attributes_completeness = []
for col in traffic_df.columns:
    attr_compl = 1-(traffic_df[col].isnull().sum()/m)
    null_attributes_completeness.append(attr_compl)

column_list = traffic_df.columns
attribute_completeness_traffic = {'attributes':list(column_list), 
                                  'attribute completeness':null_attributes_completeness}
attribute_completeness_traffic_df = pd.DataFrame(attribute_completeness_traffic, )
attribute_completeness_traffic_df

Table completeness:  0.7365121638518501 


,attributes,attribute completeness
0,description,0.982439
1,length,1.000000
2,shape,1.000000
3,speed,0.998144
4,speedUncapped,0.998144
5,freeFlow,0.999997
6,jamFactor,0.999997
7,confidence,0.998144
8,traversability,0.999997
9,subSegments,0.330634


In [16]:


#consistency of dates

dates_format = [{'format':'YYYY-MM-DD hh:mm:ss', 'collection':'WeatherData'}, 
                {'format':'YYYY-MM-DD hh:mm:ss', 'collection':'AqiAPIData'},
                {'format':'DD/MM/YYYT hh:mm:ss', 'collection':'AqiData'},
                {'format':'YYYY-MM-DDThh:mm:ssZ', 'collection':'TrafficData'}]

consistency_list = []
for d in dates_format:
    index = dates_format.index(d)
    count = 0
    for i in range(0,len(dates_format)):
        if i != index:
            if d['format'] == dates_format[i]['format']:
                count += 1
    
    cons = count/3
    consistency_list.append(cons)
    
collection_list = []
for d in dates_format:
    collection_list.append(d['collection'])
  
df_data = {'collection':collection_list, 
                                  'consistency':consistency_list}
date_consistency_df = pd.DataFrame(df_data)
date_consistency_df

,collection,consistency
0,WeatherData,0.333333
1,AqiAPIData,0.333333
2,AqiData,0.000000
3,TrafficData,0.000000


In [17]:
#consistency between the speed and the jamfactor in traffic data
cons_count = 0
err_count = 0
m = 0
for doc in document_list_traffic:
    flow = doc['currentFlow']
    if 'speed' in flow.keys() and 'freeFlow' in flow.keys() and 'jamFactor' in flow.keys():
        if flow['jamFactor'] > 0:
            m += 1
            if not(flow['speed'] > flow['freeFlow']):
                cons_count += 1
            else:
                err_count += 1
                


consistency = cons_count/m
print('Consistency of speed of veichles and jamfactor: ', consistency)
print('There are ', err_count, ' unconsistent values')

Consistency of speed of veichles and jamfactor:  0.9999569894785512
There are  8  unconsistent values


In [18]:
cons_count = 0
m = 0
for doc in document_list_traffic:
    flow = doc['currentFlow']
    if 'speed' in flow.keys() and 'speedUncapped' in flow.keys():
        m += 1
        if flow['speed'] <= flow['speedUncapped']:
            cons_count += 1
            
consistency = cons_count/m
print('Consistency of speed and speedUncapped: ', consistency)

Consistency of speed and speedUncapped:  1.0


In [19]:
cons_count = 0
m = 0
for doc in document_list_traffic:
    flow = doc['currentFlow']
    if 'jamFactor' in flow.keys() and 'traversability' in flow.keys():
        if flow['traversability'] == 'closed':
            m += 1
            if flow['jamFactor'] == 10:
                cons_count += 1
            
consistency = cons_count/m
print('Consistency of jamfactor and traversability: ', consistency)

Consistency of jamfactor and traversability:  1.0


In [20]:
#consistency between aqi and the aqi and pollutants
#consistency between all the pollutants and aqi and level
#consistency (or accuracy????) of weather data

cons_count = 0
m = 0
for doc in document_list_aqi:
    
    if doc['aqi'] != 'no data':
        m += 1
        if doc['aqi_level'] == 'Good':
            if int(doc['aqi']) <= 50:
                cons_count += 1
        elif doc['aqi_level'] == 'Moderate':
            if int(doc['aqi']) <= 100:
                cons_count += 1
        elif doc['aqi_level'] == 'Unhealthyfor Sensitive Groups':
            if int(doc['aqi']) <= 150:
                cons_count += 1
        elif doc['aqi_level'] == 'Unhealthy':
            if int(doc['aqi']) <= 200:
                cons_count += 1
        elif doc['aqi_level'] == 'VeryUnhealthy':
            if int(doc['aqi']) <= 300:
                cons_count += 1
        elif doc['aqi_level'] == 'Hazardous':
            if int(doc['aqi']) > 300:
                cons_count += 1
        
    
consistency = cons_count/m
print('Consistency of AQI and the relative category: ', consistency)

Consistency of AQI and the relative category:  1.0


In [21]:
cons_count = 0
m = 0
for doc in document_list_aqiAPI:
    pollutant_list = [doc['us_aqi_pm2_5'], doc['us_aqi_pm10']]  
    m += 1
    if doc['us_aqi'] == max(pollutant_list):
        cons_count += 1
      
consistency = cons_count/m
print('Consistency of AQI and the pollutants: ', consistency)

Consistency of AQI and the pollutants:  1.0


In [22]:
cons_count = 0
m = 0
for doc in document_list_aqi:
    
    if doc['pm 2.5'] != '-' and doc['pm 10'] != '-':
        if 'pm 1' not in doc.keys():
            pollutant_list = [int(doc['pm 2.5']), int(doc['pm 10'])]
        else:
            pollutant_list = [int(doc['pm 2.5']), int(doc['pm 10']), int(doc['pm 1'])]
            
        if doc['aqi'] != 'no data':
            m += 1
            if int(doc['aqi']) == max(pollutant_list):
                cons_count += 1
            
        
consistency = cons_count/m
print('Consistency of AQI and the pollutants: ', consistency)

Consistency of AQI and the pollutants:  0.9673333333333334


In [23]:
temp_max = 39.3
temp_min = -17.3

cons_temp_count = 0
cons_rh_count = 0
cons_wind_count = 0
m_temp = 0
m_wind = 0
m_rh = 0
for doc in document_list_aqi:
    if doc['temp'] != '-':
        m_temp += 1
        if temp_max >= int(doc['temp']) >= temp_min:
            cons_temp_count += 1
    if doc['r.h'] != '-':
        m_rh += 1
        if 100 >= int(doc['r.h']) >= 0:
            cons_rh_count += 1
    if 'wind' in doc.keys():
        m_wind += 1
        if int(doc['wind']) >= 0:
            cons_wind_count += 1

consistency_temp = cons_temp_count/m_temp
consistency_rh = cons_rh_count/m_rh
consistency_wind = cons_wind_count/m_wind

print('Accuracy of temperature in aqi data: ', consistency_temp)
print('Accuracy of relative humidity in aqi data: ', consistency_rh)
print('Accuracy of wind in aqi data: ', consistency_wind)

Accuracy of temperature in aqi data:  0.9469895287958116
Accuracy of relative humidity in aqi data:  1.0
Accuracy of wind in aqi data:  1.0


In [24]:
temp_max = 39.3
temp_min = -17.3

cons_temp_count = 0
cons_rh_count = 0
cons_wind_count = 0
cons_prcp_count = 0
m_temp = 0
m_wind = 0
m_rh = 0
m_prcp = 0
for doc in document_list_weather:
    if 'temp' in doc.keys():
        m_temp += 1
        if temp_max >= int(doc['temp']) >= temp_min:
            cons_temp_count += 1
    if 'rhum'in doc.keys():
        m_rh += 1
        if 100 >= int(doc['rhum']) >= 0:
            cons_rh_count += 1
    if 'wspd' in doc.keys():
        m_wind += 1
        if int(doc['wspd']) >= 0:
            cons_wind_count += 1
    if 'prcp' in doc.keys():
        m_prcp += 1
        if int(doc['prcp']) >= 0:
            cons_prcp_count += 1

consistency_temp = cons_temp_count/m_temp
consistency_rh = cons_rh_count/m_rh
consistency_wind = cons_wind_count/m_wind
consistency_prcp = cons_prcp_count/m_prcp

print('Accuracy of temperature in weather data: ', consistency_temp)
print('Accuracy of relative humidity in weather data: ', consistency_rh)
print('Accuracy of wind in weather data: ', consistency_wind)
print('Accuracy of precipitations in weather data: ', consistency_prcp)

Accuracy of temperature in weather data:  1.0
Accuracy of relative humidity in weather data:  1.0
Accuracy of wind in weather data:  1.0
Accuracy of precipitations in weather data:  1.0


##Cleaning and integration

##AQI

In [25]:
#remove pm1 value because the number of null
for d in document_list_aqi:  
    index = document_list_aqi.index(d)
    if 'pm 1' in d.keys():
        del d['pm 1']
        document_list_aqi[index] = d

In [26]:
#add geographical coordinates of aqi stations

for d in document_list_aqi:  
    station_id = d['station id']
    
    for cord in document_list_geocoding:
        
        if station_id == cord['station']:
            #d['location'] = {'lat': cord['lat'], 'lng': cord['lng']}
            d['location'] = (cord['lat'], cord['lng'])
            document_list_aqi[document_list_aqi.index(d)] = d

In [27]:
#delete wetather infromation from aqi data
for doc in document_list_aqi:
    if 'pressure' in doc.keys():
        del doc['pressure']
    if 'temp' in doc.keys():
        del doc['temp']
    if 'r.h' in doc.keys():
        del doc['r.h']
    if 'wind' in doc.keys():
        del doc['wind']
    
    

##WEATHER DATA

In [28]:
#remove tsun and snow because the numbers of null values
for doc in document_list_weather:
    if 'snow' in doc.keys():
        del doc['snow']
    if 'tsun' in doc.keys():
        del doc['tsun']
    
    

##TRAFFIC DATA

In [29]:
#convert from m/s to km/h
index = 0
for doc in document_list_traffic:
    
    currentFlow = doc['currentFlow']
    
    if 'speed' in currentFlow.keys():
        currentFlow['speed'] = currentFlow['speed'] * 3.6
        
    if 'speedUncapped' in currentFlow.keys():
        currentFlow['speedUncapped'] = currentFlow['speedUncapped'] * 3.6
        
    if 'freeFlow' in currentFlow.keys():
        currentFlow['freeFlow'] = currentFlow['freeFlow'] * 3.6
    
    document_list_traffic[index]['currentFlow'] = currentFlow
    index += 1 

In [30]:
#computing the first and the last point of the street from the shape
index = 0
for doc in document_list_traffic:
    loc = doc['location']
    shape = loc['shape']
    links = shape['links']
    
    first_link = links[0]
    first_point = (first_link['points'][0]['lat'], first_link['points'][0]['lng'])
    
    last_link = links[-1]
    last_point = (last_link['points'][-1]['lat'], last_link['points'][-1]['lng'])
    document_list_traffic[index]['location']['start point'] = first_point
    document_list_traffic[index]['location']['end point'] = last_point
    index += 1

In [31]:
#definetly remove the encoding error
for doc in document_list_traffic:
    if 'description' in doc['location'].keys():
        if 'Ã' in doc['location']['description']:
            desc = doc['location']['description']
            desc = desc[0:desc.find('Ã')]
            document_list_traffic[document_list_traffic.index(doc)]['location']['description'] = desc
    

In [32]:
#removing null attributes
for doc in document_list_traffic:
    flow = doc['currentFlow']
    location = doc['location']
    
    if 'junctionTraversability' in flow.keys():
        del doc['currentFlow']['junctionTraversability']
    if 'jamTendency' in flow.keys():
        del doc['currentFlow']['jamTendency']
    if 'no infos' in flow.keys():
        del doc['currentFlow']['no infos']
    if 'subSegments' in flow.keys():
        del doc['currentFlow']['subSegments']

In [33]:
#remove record with null traffic infos
for doc in document_list_traffic:
    flow = doc['currentFlow']
    if 'speed' not in flow.keys() or 'speedUncapped' not in flow.keys() or 'speedUncapped' not in flow.keys() or 'jamFactor' not in flow.keys() or 'confidence' not in flow.keys():
        index = document_list_traffic.index(doc)
        del document_list_traffic[index]

In [34]:
#remove records with null location description
for doc in document_list_traffic:
   location = doc['location']
   if 'description' not in location.keys():
       index = document_list_traffic.index(doc)
       del document_list_traffic[index]

In [35]:
#fix the inconsistency between jamfactor and speed
for doc in document_list_traffic:
    
    flow = doc['currentFlow']
    if 'speed' in flow.keys() and 'freeFlow' in flow.keys() and 'jamFactor' in flow.keys():
        if flow['jamFactor'] > 0:
            if flow['speed'] > flow['freeFlow']:
                document_list_traffic[document_list_traffic.index(doc)]['currentFlow']['jamFactor'] = 0

##STANDARDIZE DATETIME FORMAT AND TIMES

In [36]:

index = 0
for doc in document_list_aqi:
    time = doc['time']
    std_time = time[11:13] + ':00:00'
    date = time[0:10]
    date = date.replace('/', '-')
    datetime = date +' '+ std_time
    document_list_aqi[index]['time'] = datetime
    index += 1

In [37]:
index = 0
for doc in document_list_traffic:
    datetime = doc['datetime']
    if datetime[14:15] == '5':
        if datetime[11:13] == '23':
            new_hour = str(int(datetime[11:13])+1)
            new_min = '00'
            new_day = str(int(datetime[8:10])+1)
            if len(new_day) == 1:
                new_day = '0'+new_day
            if len(new_hour) == 1:
                new_hour = '0'+new_hour
            new_dt = datetime[:8]+new_day+'T'+new_hour+':'+new_min+datetime[16:]
            document_list_traffic[index]['datetime'] = new_dt
        else:
            new_hour = str(int(datetime[11:13])+1)
            new_min = '00'
            if len(new_hour) == 1:
                new_hour = '0'+new_hour
            new_dt = datetime[:11]+new_hour+':'+new_min+datetime[16:]
            document_list_traffic[index]['datetime'] = new_dt
    index += 1
        

In [38]:
index = 0
for doc in document_list_traffic:
    datetime = doc['datetime']
    date = datetime[:10]
    year = date[:4]
    month = date[5:7]
    day = date[8:10]
    new_date = day+'-'+month+'-'+year
    time = datetime[11:]
    time = time[:2]+':00:00'
    document_list_traffic[index]['datetime'] = new_date+' '+time
    index +=1

In [39]:
index = 0
for doc in document_list_weather:
    datetime = doc['datetime']
    date = datetime[:10]
    year = date[:4]
    month = date[5:7]
    day = date[8:10]
    new_date = day+'-'+month+'-'+year
    time = datetime[11:]
    new_datetime = new_date+' '+time
    document_list_weather[index]['datetime'] = new_datetime
    index +=1

In [40]:
index = 0
for doc in document_list_aqiAPI:
    datetime = doc['datetime']
    date = datetime[:10]
    year = date[:4]
    month = date[5:7]
    day = date[8:10]
    new_date = day+'-'+month+'-'+year
    time = datetime[11:]
    new_datetime = new_date+' '+time
    document_list_aqiAPI[index]['datetime'] = new_datetime
    index += 1

##DATA INTEGRATION

In [41]:
df_weather = pd.DataFrame.from_records(document_list_weather)
df_weather

,_id,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,coco,datetime,station
0,65cdfe48adcb82fd86ce92a3,4.0,2.4,89.0,0.0,0.0,0.0,5.5,1028.3,2.0,03-02-2024 00:00:00,98497
1,65cdfe48adcb82fd86ce92a4,3.3,1.3,87.0,0.0,0.0,2.0,5.5,1028.2,2.0,03-02-2024 01:00:00,98497
2,65cdfe48adcb82fd86ce92a5,3.5,2.6,94.0,0.0,0.0,2.0,5.5,1027.7,2.0,03-02-2024 02:00:00,98497
3,65cdfe48adcb82fd86ce92a6,2.6,1.7,94.0,0.0,30.0,4.0,3.7,1027.4,1.0,03-02-2024 03:00:00,98497
4,65cdfe48adcb82fd86ce92a7,2.5,1.8,95.0,0.0,0.0,0.0,7.4,1027.4,1.0,03-02-2024 04:00:00,98497
...,...,...,...,...,...,...,...,...,...,...,...,...
4092,65cdfe4aadcb82fd86cea29f,5.5,5.5,100.0,0.0,360.0,5.0,9.3,1009.5,2.0,12-02-2024 20:00:00,193501
4093,65cdfe4aadcb82fd86cea2a0,6.5,6.5,100.0,0.0,350.0,5.0,9.3,1010.4,2.0,12-02-2024 21:00:00,193501
4094,65cdfe4aadcb82fd86cea2a1,6.0,5.7,98.0,0.0,320.0,4.0,9.3,1011.1,2.0,12-02-2024 22:00:00,193501
4095,65cdfe4aadcb82fd86cea2a2,6.2,6.2,100.0,0.0,0.0,2.0,9.3,1011.7,2.0,12-02-2024 23:00:00,193501


In [42]:
df_aqi = pd.DataFrame.from_records(document_list_aqi)
df_aqi

,_id,station id,time,aqi,aqi_level,pm 2.5,pm 10,location
0,65cdfe48adcb82fd86ce85f4,98497,04-02-2024 00:00:00,51,Moderate,51,19,"(45.5452297, 9.2336167)"
1,65cdfe48adcb82fd86ce85f5,107293,04-02-2024 00:00:00,37,Good,37,15,"(45.5540815, 9.2603531)"
2,65cdfe48adcb82fd86ce85f6,98494,04-02-2024 00:00:00,33,Good,33,11,"(45.5396143, 9.2365339)"
3,65cdfe48adcb82fd86ce85f7,98485,04-02-2024 00:00:00,42,Good,42,22,"(45.5255896, 9.2285405)"
4,65cdfe48adcb82fd86ce85f8,460336,04-02-2024 00:00:00,89,Moderate,89,29,"(45.496831549999996, 9.167650701677125)"
...,...,...,...,...,...,...,...,...
3242,65cdfe48adcb82fd86ce929e,116353,09-02-2024 08:00:00,43,Good,43,23,"(45.4679729, 9.1235389)"
3243,65cdfe48adcb82fd86ce929f,246697,09-02-2024 08:00:00,62,Moderate,62,38,"(45.4429376, 9.2109626)"
3244,65cdfe48adcb82fd86ce92a0,196504,09-02-2024 08:00:00,88,Moderate,88,44,"(45.4394463, 9.178421)"
3245,65cdfe48adcb82fd86ce92a1,74863,09-02-2024 08:00:00,95,Moderate,95,51,"(45.4265036, 9.2076193)"


In [43]:
location_list = []
for d in document_list_traffic:
    location_list.append(d['location'])

In [44]:
traffic_list = []
for d in document_list_traffic:
    traffic_list.append(d['currentFlow'])

In [45]:
datetime_list = []
for d in document_list_traffic:
    datetime_list.append(d['datetime'])

In [46]:
location_df = pd.DataFrame.from_records(location_list)
flow_df = pd.DataFrame.from_records(traffic_list)

In [47]:
location_df_unique = location_df.drop_duplicates(subset=['description', 'start point', 'end point'])
location_df_unique.head()

,description,length,shape,start point,end point
0,Uscita 5 - Palazzolo Milanese,403.0,"{'links': [{'points': [{'lat': 45.57636, 'lng'...","(45.57636, 9.17601)","(45.57998, 9.1755)"
1,SP13-Viale Sicilia,46.0,"{'links': [{'points': [{'lat': 45.5809, 'lng':...","(45.5809, 9.3098)","(45.58124, 9.30994)"
2,Muggio',2350.0,"{'links': [{'points': [{'lat': 45.592, 'lng': ...","(45.592, 9.19793)","(45.59047, 9.2268)"
3,Piazzale Giulio Cesare,394.0,"{'links': [{'points': [{'lat': 45.47055, 'lng'...","(45.47055, 9.15555)","(45.47352, 9.15519)"
4,SP103-Segrate,2656.0,"{'links': [{'points': [{'lat': 45.47797, 'lng'...","(45.47797, 9.30407)","(45.4978, 9.30095)"


In [48]:
location_df_unique[location_df_unique['description'].isnull()]


,description,length,shape,start point,end point
29991,NaN,33.0,"{'links': [{'points': [{'lat': 45.39042, 'lng'...","(45.39042, 9.1726)","(45.39013, 9.17254)"
32134,NaN,284.0,"{'links': [{'points': [{'lat': 45.55712, 'lng'...","(45.55712, 9.26737)","(45.55719, 9.26388)"
32207,NaN,101.0,"{'links': [{'points': [{'lat': 45.5556, 'lng':...","(45.5556, 9.2642)","(45.5553, 9.26299)"
35227,NaN,58.0,"{'links': [{'points': [{'lat': 45.55719, 'lng'...","(45.55719, 9.26388)","(45.55713, 9.26314)"
49007,NaN,789.0,"{'links': [{'points': [{'lat': 45.49665, 'lng'...","(45.49665, 9.17119)","(45.50237, 9.16557)"
...,...,...,...,...,...
353123,NaN,262.0,"{'links': [{'points': [{'lat': 45.54816, 'lng'...","(45.54816, 9.18609)","(45.54771, 9.1829)"
353388,NaN,326.0,"{'links': [{'points': [{'lat': 45.56874, 'lng'...","(45.56874, 9.24504)","(45.56593, 9.24384)"
353656,NaN,1248.0,"{'links': [{'points': [{'lat': 45.5367, 'lng':...","(45.5367, 9.30522)","(45.52634, 9.31104)"
353699,NaN,367.0,"{'links': [{'points': [{'lat': 45.51367, 'lng'...","(45.51367, 9.03284)","(45.51436, 9.03737)"


In [49]:
import geopy.distance


associated_stations = []
for index,loc in location_df_unique.iterrows():
    start = loc['start point']
    end = loc['end point']
    station_list = []
    distances_list = []
    for d in document_list_geocoding:
        point = (d['lat'], d['lng'])
        station = d['station']
        station_list.append(station)
        d1 = geopy.distance.geodesic(start, point).km
        d2 = geopy.distance.geodesic(end, point).km
        if d1 < d2:
            distances_list.append(d1)
        else:
            distances_list.append(d2)
    
    associated_stations.append(station_list[distances_list.index(min(distances_list))])
    
            

In [50]:
location_df_unique['station'] = associated_stations

In [51]:
location_df = pd.merge(location_df, location_df_unique, left_on=['description', 'start point', 'end point'], right_on=['description', 'start point', 'end point'])

In [52]:
location_df = location_df.drop(['length_y', 'shape_y'], axis=1)
location_df = location_df.rename({'length_x' : 'length', 'shape_x':'shape'}, axis=1)

In [53]:
traffic_df = pd.merge(location_df, flow_df, right_index=True, left_index=True)
traffic_df['datetime'] = datetime_list
ind_list = list(traffic_df[traffic_df['description'].isnull()].index)
traffic_df = traffic_df.drop(ind_list)

In [54]:
traffic_df.head()

,description,length,shape,start point,end point,station,speed,speedUncapped,freeFlow,jamFactor,confidence,traversability,datetime
0,Uscita 5 - Palazzolo Milanese,403.0,"{'links': [{'points': [{'lat': 45.57636, 'lng'...","(45.57636, 9.17601)","(45.57998, 9.1755)",98497,89.000003,89.000003,82.999998,0.0,0.99,open,04-02-2024 24:00:00
1,SP13-Viale Sicilia,46.0,"{'links': [{'points': [{'lat': 45.5809, 'lng':...","(45.5809, 9.3098)","(45.58124, 9.30994)",107293,41.000000,41.000000,38.000002,0.0,0.99,open,04-02-2024 24:00:00
2,Muggio',2350.0,"{'links': [{'points': [{'lat': 45.592, 'lng': ...","(45.592, 9.19793)","(45.59047, 9.2268)",107293,43.999999,43.999999,38.000002,0.0,0.85,open,04-02-2024 24:00:00
3,Piazzale Giulio Cesare,394.0,"{'links': [{'points': [{'lat': 45.47055, 'lng'...","(45.47055, 9.15555)","(45.47352, 9.15519)",69364,19.000001,19.000001,28.000001,1.9,0.98,open,04-02-2024 24:00:00
4,SP103-Segrate,2656.0,"{'links': [{'points': [{'lat': 45.47797, 'lng'...","(45.47797, 9.30407)","(45.4978, 9.30095)",156526,43.000002,43.000002,41.000000,0.0,0.86,open,04-02-2024 24:00:00


In [55]:
df_aqi_API = pd.DataFrame.from_records(document_list_aqiAPI)
df_aqi_API.head()

,_id,us_aqi,us_aqi_pm2_5,us_aqi_pm10,us_aqi_nitrogen_dioxide,us_aqi_carbon_monoxide,us_aqi_ozone,us_aqi_sulphur_dioxide,station,datetime
0,65ce00ebadcb82fd86d422c6,159.921051,159.921051,63.514584,29.550827,2.817633,0.502474,3.053435,98497,02-02-2024 23:00:00
1,65ce00ebadcb82fd86d422c7,159.771912,159.771912,63.443748,26.004728,2.977858,0.077304,2.290076,98497,03-02-2024 00:00:00
2,65ce00ebadcb82fd86d422c8,159.633774,159.633774,63.289581,21.424351,3.034219,0.000000,1.472192,98497,03-02-2024 01:00:00
3,65ce00ebadcb82fd86d422c9,159.449570,159.449570,63.072922,17.779749,3.004831,0.038652,1.035987,98497,03-02-2024 02:00:00
4,65ce00ebadcb82fd86d422ca,159.085526,159.085526,62.770832,14.036644,2.853462,0.115955,1.035987,98497,03-02-2024 03:00:00


In [56]:
df_aqi = df_aqi.drop('_id', axis=1)
df_aqi_API = df_aqi_API.drop('_id', axis=1)

In [57]:
df_aqi = pd.merge(df_aqi_API, df_aqi, left_on=['station', 'datetime'], right_on=['station id', 'time'])
df_aqi.head()

,us_aqi,us_aqi_pm2_5,us_aqi_pm10,us_aqi_nitrogen_dioxide,us_aqi_carbon_monoxide,us_aqi_ozone,us_aqi_sulphur_dioxide,station,datetime,station id,time,aqi,aqi_level,pm 2.5,pm 10,location
0,127.697906,127.697906,46.909096,19.700552,1.478663,1.700680,1.417666,98497,04-02-2024 00:00:00,98497,04-02-2024 00:00:00,51,Moderate,51,19,"(45.5452297, 9.2336167)"
1,123.520828,123.520828,45.632576,19.651300,1.545089,1.178881,1.526718,98497,04-02-2024 01:00:00,98497,04-02-2024 01:00:00,51,Moderate,51,17,"(45.5452297, 9.2336167)"
2,120.677101,120.677101,44.579548,15.120174,1.583736,1.024273,1.308615,98497,04-02-2024 02:00:00,98497,04-02-2024 02:00:00,50,Moderate,50,19,"(45.5452297, 9.2336167)"
3,119.552086,119.552086,44.181820,14.972419,1.580515,0.888992,1.199564,98497,04-02-2024 03:00:00,98497,04-02-2024 03:00:00,52,Moderate,52,17,"(45.5452297, 9.2336167)"
4,119.989601,119.989601,44.363640,15.957448,1.563205,0.908318,1.254089,98497,04-02-2024 04:00:00,98497,04-02-2024 04:00:00,54,Moderate,54,20,"(45.5452297, 9.2336167)"


In [58]:
df_environment = pd.merge(df_weather, df_aqi, left_on=['station', 'datetime'], right_on=['station id', 'datetime'])


In [59]:
df_environment = df_environment.drop(['_id', 'dwpt', 'wdir', 'wpgt', 'coco', 'time', 'station_y', 'station_x'], axis=1)
df_environment.head()

,temp,rhum,prcp,wspd,pres,datetime,us_aqi,us_aqi_pm2_5,us_aqi_pm10,us_aqi_nitrogen_dioxide,us_aqi_carbon_monoxide,us_aqi_ozone,us_aqi_sulphur_dioxide,station id,aqi,aqi_level,pm 2.5,pm 10,location
0,9.0,81.0,0.0,4.0,1024.1,04-02-2024 00:00:00,127.697906,127.697906,46.909096,19.700552,1.478663,1.700680,1.417666,98497,51,Moderate,51,19,"(45.5452297, 9.2336167)"
1,8.5,84.0,0.0,8.0,1023.7,04-02-2024 01:00:00,123.520828,123.520828,45.632576,19.651300,1.545089,1.178881,1.526718,98497,51,Moderate,51,17,"(45.5452297, 9.2336167)"
2,6.3,88.0,0.0,4.0,1023.8,04-02-2024 02:00:00,120.677101,120.677101,44.579548,15.120174,1.583736,1.024273,1.308615,98497,50,Moderate,50,19,"(45.5452297, 9.2336167)"
3,6.1,95.0,0.0,5.0,1023.9,04-02-2024 03:00:00,119.552086,119.552086,44.181820,14.972419,1.580515,0.888992,1.199564,98497,52,Moderate,52,17,"(45.5452297, 9.2336167)"
4,4.0,96.0,0.0,2.0,1023.3,04-02-2024 04:00:00,119.989601,119.989601,44.363640,15.957448,1.563205,0.908318,1.254089,98497,54,Moderate,54,20,"(45.5452297, 9.2336167)"


In [60]:
for index, row in df_environment.iterrows():
    if row['aqi'] == 'no data':
        row['aqi'] = row['us_aqi']
        if int(row['aqi']) < 100:
            row['aqi_level'] = 'Moderate'
        elif int(row['aqi']) < 150:
            row['aqi_level'] = 'Unhealthy for Sensitive Groups'
        elif int(row['aqi']) < 200:
            row['aqi_level'] = 'Unhealthy'
        elif int(row['aqi']) < 300:
            row['aqi_level'] = 'Very Unhealthy'
        else:
            row['aqi_level'] = 'Hazardous'
    if row['pm 2.5'] == '-':
        row['pm 2.5'] = row['us_aqi_pm2_5']
    if row['pm 10'] == '-':
        row['pm 10'] = row['us_aqi_pm10']
    
    df_environment.loc[index] = row

In [61]:
for index, row in df_environment.iterrows():
     if row['pm 2.5'] != '-' and row['pm 10'] != '-':
        pollutant_list = [int(row['pm 2.5']), int(row['pm 10'])]
        
        if int(row['aqi']) != max(pollutant_list):
            row['aqi'] = row['us_aqi']
            if int(row['aqi']) < 100:
                row['aqi_level'] = 'Moderate'
            elif int(row['aqi']) < 150:
                row['aqi_level'] = 'Unhealthy for Sensitive Groups'
            elif int(row['aqi']) < 200:
                row['aqi_level'] = 'Unhealthy'
            elif int(row['aqi']) < 300:
                row['aqi_level'] = 'Very Unhealthy'
            else:
                row['aqi_level'] = 'Hazardous'
            
            row['pm 2.5'] = row['us_aqi_pm2_5']
            row['pm 10'] = row['us_aqi_pm10']
            df_environment.loc[index] = row

In [62]:
df_environment = df_environment.drop(['us_aqi', 'us_aqi_pm2_5', 'us_aqi_pm10'], axis=1)
df_environment = df_environment.rename({'us_aqi_carbon_monoxide': 'CO', 'us_aqi_nitrogen_dioxide': 'NO2',
                                        'us_aqi_ozone': 'O3', 'us_aqi_sulphur_dioxide': 'SO2', 
                                        'location': 'station location'}, axis=1)

In [63]:
df_environment

,temp,rhum,prcp,wspd,pres,datetime,NO2,CO,O3,SO2,station id,aqi,aqi_level,pm 2.5,pm 10,station location
0,9.0,81.0,0.0,4.0,1024.1,04-02-2024 00:00:00,19.700552,1.478663,1.700680,1.417666,98497,51,Moderate,51,19,"(45.5452297, 9.2336167)"
1,8.5,84.0,0.0,8.0,1023.7,04-02-2024 01:00:00,19.651300,1.545089,1.178881,1.526718,98497,51,Moderate,51,17,"(45.5452297, 9.2336167)"
2,6.3,88.0,0.0,4.0,1023.8,04-02-2024 02:00:00,15.120174,1.583736,1.024273,1.308615,98497,50,Moderate,50,19,"(45.5452297, 9.2336167)"
3,6.1,95.0,0.0,5.0,1023.9,04-02-2024 03:00:00,14.972419,1.580515,0.888992,1.199564,98497,52,Moderate,52,17,"(45.5452297, 9.2336167)"
4,4.0,96.0,0.0,2.0,1023.3,04-02-2024 04:00:00,15.957448,1.563205,0.908318,1.254089,98497,54,Moderate,54,20,"(45.5452297, 9.2336167)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,8.9,95.0,0.0,2.0,997.4,11-02-2024 19:00:00,12.509850,0.694042,7.285869,1.254089,193501,12,Good,12,6,"(45.4452809, 9.0937168)"
3243,8.6,97.0,0.0,0.0,997.9,11-02-2024 20:00:00,13.544128,0.750805,6.976655,1.090513,193501,32,Good,32,11,"(45.4452809, 9.0937168)"
3244,7.4,99.0,0.0,2.0,998.6,11-02-2024 21:00:00,15.070923,0.838567,6.338900,1.308615,193501,35,Good,35,12,"(45.4452809, 9.0937168)"
3245,7.8,100.0,0.0,2.0,999.1,11-02-2024 22:00:00,14.479905,0.952899,5.507885,1.635769,193501,37,Good,37,14,"(45.4452809, 9.0937168)"


In [64]:
df = pd.merge(traffic_df, df_environment, left_on=['station', 'datetime'], right_on=['station id', 'datetime'])

In [65]:
df = df.drop(['shape', 'confidence', 'station'], axis=1)
df.head()

,description,length,start point,end point,speed,speedUncapped,freeFlow,jamFactor,traversability,datetime,...,NO2,CO,O3,SO2,station id,aqi,aqi_level,pm 2.5,pm 10,station location
0,Via Eritrea,1029.0,"(45.50902, 9.14808)","(45.51075, 9.1376)",40.000003,40.000003,34.999999,0.0,open,04-02-2024 01:00:00,...,10.983057,1.152174,2.744279,0.763359,460336,87,Moderate,87,28,"(45.496831549999996, 9.167650701677125)"
1,Lainate,1862.0,"(45.57173, 9.01591)","(45.56162, 9.03485)",108.000000,108.000000,105.000005,0.0,open,04-02-2024 01:00:00,...,10.983057,1.152174,2.744279,0.763359,116353,25,Good,25,10,"(45.4679729, 9.1235389)"
2,A4 - Torino Trieste,97.0,"(45.51447, 9.09862)","(45.51419, 9.09977)",64.999998,64.999998,40.000003,0.0,open,04-02-2024 01:00:00,...,10.983057,1.152174,2.744279,0.763359,116353,25,Good,25,10,"(45.4679729, 9.1235389)"
3,SP114-Cusago,3716.0,"(45.41276, 9.03626)","(45.44315, 9.02899)",72.000000,72.000000,50.000000,0.0,open,04-02-2024 01:00:00,...,9.653271,1.156200,3.575294,0.817885,193501,32,Good,32,12,"(45.4452809, 9.0937168)"
4,Via Luigi Nono incrocio Via Giulio Cesare Proc...,329.0,"(45.48675, 9.17499)","(45.48409, 9.17685)",34.999999,34.999999,30.000002,0.0,open,04-02-2024 01:00:00,...,10.983057,1.152174,2.744279,0.763359,21373,73,Moderate,73,22,"(45.4762915, 9.1721766)"


In [66]:
df[['aqi', 'pm 2.5', 'pm 10']] = df[['aqi', 'pm 2.5', 'pm 10']].astype(int)

In [67]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M:%S')

In [68]:
wd_list = []
for dt in df['datetime']:
    wd_list.append(dt.weekday())
df['weekday'] = wd_list

In [69]:
df['speedGap'] = df['speedUncapped']-df['freeFlow']

In [70]:
df.head()

,description,length,start point,end point,speed,speedUncapped,freeFlow,jamFactor,traversability,datetime,...,O3,SO2,station id,aqi,aqi_level,pm 2.5,pm 10,station location,weekday,speedGap
0,Via Eritrea,1029.0,"(45.50902, 9.14808)","(45.51075, 9.1376)",40.000003,40.000003,34.999999,0.0,open,2024-02-04 01:00:00,...,2.744279,0.763359,460336,87,Moderate,87,28,"(45.496831549999996, 9.167650701677125)",6,5.000004
1,Lainate,1862.0,"(45.57173, 9.01591)","(45.56162, 9.03485)",108.000000,108.000000,105.000005,0.0,open,2024-02-04 01:00:00,...,2.744279,0.763359,116353,25,Good,25,10,"(45.4679729, 9.1235389)",6,2.999995
2,A4 - Torino Trieste,97.0,"(45.51447, 9.09862)","(45.51419, 9.09977)",64.999998,64.999998,40.000003,0.0,open,2024-02-04 01:00:00,...,2.744279,0.763359,116353,25,Good,25,10,"(45.4679729, 9.1235389)",6,24.999995
3,SP114-Cusago,3716.0,"(45.41276, 9.03626)","(45.44315, 9.02899)",72.000000,72.000000,50.000000,0.0,open,2024-02-04 01:00:00,...,3.575294,0.817885,193501,32,Good,32,12,"(45.4452809, 9.0937168)",6,22.000000
4,Via Luigi Nono incrocio Via Giulio Cesare Proc...,329.0,"(45.48675, 9.17499)","(45.48409, 9.17685)",34.999999,34.999999,30.000002,0.0,open,2024-02-04 01:00:00,...,2.744279,0.763359,21373,73,Moderate,73,22,"(45.4762915, 9.1721766)",6,4.999997


In [71]:
count = 0
for e in df['description'].isnull():
    if e:
        print(df.iloc[count])
    count+=1

In [72]:

m = len(df)
n = len(list(df.columns))
#table completeness
null_table = 0
for col in df.columns:
    null_table += df[col].isnull().sum()
table_completeness = 1-(null_table/(m*n))    
print('Table completeness: ', table_completeness, '\n\n')

#attribute completeness
null_attributes_completeness = []
for col in df.columns:
    attr_compl = 1-(df[col].isnull().sum()/m)
    null_attributes_completeness.append(attr_compl)
    
            

column_list = df.columns
attribute_completeness = {'attributes':list(column_list), 
                                  'attribute completeness':null_attributes_completeness}
attribute_completeness_df = pd.DataFrame(attribute_completeness)
attribute_completeness_df

Table completeness:  1.0 


,attributes,attribute completeness
0,description,1.0
1,length,1.0
2,start point,1.0
3,end point,1.0
4,speed,1.0
5,speedUncapped,1.0
6,freeFlow,1.0
7,jamFactor,1.0
8,traversability,1.0
9,datetime,1.0


In [73]:
cons_count = 0
m = 0

for index, row in df.iterrows():
    pollutant_list = [int(row['pm 2.5']), int(row['pm 10'])]
    m += 1
    if int(row['aqi']) == max(pollutant_list):
        cons_count += 1

consistency = cons_count/m
print('consistency for aqi and pollutants: ', consistency)

consistency for aqi and pollutants:  1.0


In [74]:
cons_count = 0
m = 0
for index, row in df.iterrows():
   if row['jamFactor'] > 0:
        m += 1
        if not(row['speed'] > row['freeFlow']):
            cons_count += 1
                

consistency = cons_count/m
print('consistency for speed and jamFactor: ', consistency)

consistency for speed and jamFactor:  1.0


##UPLOAD CLEANED AND AGGREGATED DATA ON DB

In [75]:

result_document_list = []
for index, row in df.iterrows():
    station = {'id':row['station id'], 'station location': row['station location']}
    location = {'description': row['description'], 'length':row['length'], 
                'start point':row['start point'], 'end point': row['end point'], 'station':station}
    traffic = {'speed':row['speed'],'speedUncapped':row['speedUncapped'],
           'freeFlow':row['freeFlow'], 'jamFactor':row['jamFactor'], 'traversability':row['traversability'], 'speedGap':row['speedGap']}
    environment = {'temperature': row['temp'], 'relative humidity': row['rhum'], 'rainfall': row['prcp'],
                   'wind': row['wspd'],'pressure': row['pres'], 'NO2': row['NO2'], 'CO': row['CO'], 
                   'O3': row['O3'],'SO2': row['SO2'],'Aqi': row['aqi'],'aqi level': row['aqi_level'],
                   'pm 2.5': row['pm 2.5'],'pm 10': row['pm 10'],}
    doc = {'datetime':row['datetime'], 'weekday': row['weekday'],'environment': environment, 'traffic':traffic, 'location': location}
    result_document_list.append(doc)
    

In [76]:
clean_db = client.DataDb
collection = clean_db.MilanTrafficAndPollution
print(result_document_list[0])
for doc in result_document_list:
    collection.insert_one(doc)


{'datetime': Timestamp('2024-02-04 01:00:00'), 'weekday': 6, 'environment': {'temperature': 8.5, 'relative humidity': 84.0, 'rainfall': 0.0, 'wind': 8.0, 'pressure': 1023.7, 'NO2': 10.983057022094727, 'CO': 1.1521738767623901, 'O3': 2.744279384613037, 'SO2': 0.7633588314056396, 'Aqi': 87, 'aqi level': 'Moderate', 'pm 2.5': 87, 'pm 10': 28}, 'traffic': {'speed': 40.0000032, 'speedUncapped': 40.0000032, 'freeFlow': 34.999999200000005, 'jamFactor': 0.0, 'traversability': 'open', 'speedGap': 5.000003999999997}, 'location': {'description': 'Via Eritrea', 'length': 1029.0, 'start point': (45.50902, 9.14808), 'end point': (45.51075, 9.1376), 'station': {'id': 460336, 'station location': (45.496831549999996, 9.167650701677125)}}}


In [77]:

environment_doc_list = []
traffic_doc_list = []
for index, row in df.iterrows():
    station = {'id':row['station id'], 'station location': row['station location']}
    location = {'description': row['description'], 'length':row['length'], 
                'start point':row['start point'], 'end point': row['end point'], 'station':station}
    traffic = {'speed':row['speed'],'speedUncapped':row['speedUncapped'],
           'freeFlow':row['freeFlow'], 'jamFactor':row['jamFactor'], 'traversability':row['traversability'], 'speedGap':row['speedGap']}
    environment = {'temperature': row['temp'], 'relative humidity': row['rhum'], 'rainfall': row['prcp'],
                   'wind': row['wspd'],'pressure': row['pres'], 'NO2': row['NO2'], 'CO': row['CO'], 
                   'O3': row['O3'],'SO2': row['SO2'],'Aqi': row['aqi'],'aqi level': row['aqi_level'],
                   'pm 2.5': row['pm 2.5'],'pm 10': row['pm 10'],}
    environment_tmp = environment
    environment_tmp['location'] = location
    environment_tmp['datetime'] = row['datetime']
    environment_tmp['station'] = row['station id']
    environment_doc_list.append(environment_tmp)
    traffic_tmp = traffic
    traffic_tmp['location'] = location
    traffic_tmp['datetime'] = row['datetime']
    traffic_doc_list.append(traffic_tmp)

In [78]:
clean_db = client.DataDb
collection = clean_db.EnvironmentData

df_tmp = pd.DataFrame.from_records(environment_doc_list)
df_tmp = df_tmp.drop_duplicates(subset=['datetime', 'station'])
environment_doc_list = []
for index, row in df_tmp.iterrows():
    environment = {'temperature': row['temperature'], 'relative humidity': row['temperature'], 'rainfall': row['rainfall'],
                'wind': row['wind'],'pressure': row['pressure'], 'NO2': row['NO2'], 'CO': row['CO'], 'O3': row['O3'],
                'SO2': row['SO2'],'Aqi': row['Aqi'],'aqi level': row['aqi level'],'pm 2.5': row['pm 2.5'],
                'pm 10': row['pm 10'],}
    environment_tmp = environment
    environment_tmp['location'] = row['location']
    environment_tmp['datetime'] = row['datetime']
    environment_doc_list.append(environment_tmp)


for doc in environment_doc_list:
    collection.insert_one(doc)

In [79]:
clean_db = client.DataDb
collection = clean_db.TrafficData

for doc in traffic_doc_list:
    collection.insert_one(doc)